<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [134]:
import pandas as pd
from tensorflow import keras
from keras.optimizers import SGD
from keras.models import Sequential 
from keras.layers import Dense, Dropout
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

df = pd.read_csv('Customer.csv')

In [60]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [61]:
def cat_encode(X):
    
    X = X.copy()

    # import labelencoder
    from sklearn.preprocessing import LabelEncoder
    
    le = LabelEncoder()

    #Categorical boolean mask
    categorical_feature_mask = X.dtypes==object
    categorical_cols = X.columns[categorical_feature_mask].tolist()

    X[categorical_cols] = X[categorical_cols].apply(lambda col: le.fit_transform(col))
    
    return X

In [62]:
df = cat_encode(df)

In [63]:
target = 'Churn'
features = ['gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges']

X = df[features]
y = df[target]

In [77]:
#Identify target in our df and set the predicted value to the mode
y_true = df['Churn']
majority = df['Churn'].mode()[0]

#Create a list of predictions for the length of our df
y_pred = [majority] * len(y_true)

#Use accuracy_score to check our baseline accuracy
from sklearn.metrics import accuracy_score
print("Baseline Accuracy:", accuracy_score(y_true, y_pred))

Baseline Accuracy: 0.7346301292063041


In [64]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    X, 
    y,
    test_size=0.2)

x_train.shape, x_test.shape, y_train.shape, y_test.shape

((5634, 19), (1409, 19), (5634,), (1409,))

In [124]:
batch_size = 20
num_classes = 2
epochs = 10

In [125]:
opt = SGD(lr=0.1, momentum=0.9)

In [135]:
model = Sequential()
model.add(Dense(5, activation='relu', input_shape=(19,)))
# model.add(Dropout(0.2))
model.add(Dense(3, activation='relu'))
model.add(Dense(3, activation='relu'))
# model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(
    loss='binary_crossentropy', 
    optimizer=opt, 
    metrics=['accuracy']
            )

history = model.fit(
    x_train, 
    y_train, 
    validation_data=(x_test, y_test), 
    epochs=epochs,
    batch_size=batch_size,
    verbose=0)

In [136]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test accuracy:', score[1])

Test accuracy: 0.7515968680381775


In [154]:
def create_model():
    opt = SGD(lr=0.1, momentum=0.9)
    model = Sequential()
    model.add(Dense(5, activation='tanh', input_shape=(19,)))
    # model.add(Dropout(0.2))
    model.add(Dense(3, activation='relu'))
    model.add(Dense(3, activation='relu'))
    # model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
param_grid = {'batch_size': [2],
              'epochs': [30]}

In [155]:
# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(x_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.730387806892395 using {'batch_size': 2, 'epochs': 30}
Means: 0.730387806892395, Stdev: 0.012597235182934968 with: {'batch_size': 2, 'epochs': 30}


## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?